In [1]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os

In [5]:
# 1. 대상 URL과 헤더 설정
url = "https://www.moviechart.co.kr/rank/realtime/index/image"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# 2. 페이지 요청
movie_ranking = requests.get(url, headers=headers)

if movie_ranking.status_code == 200:
    print('✅ 페이지를 성공적으로 불러왔습니다!')
    
    # .content를 사용하여 바이트 데이터를 BeautifulSoup에 전달 (인코딩에 유리)
    soup = BeautifulSoup(movie_ranking.content, 'html.parser')

    # 3. 영화 리스트가 담긴 요소 찾기 
    #content > div.wArea.space > div.movieBox > ul > li:nth-child(1)
    # (선택자에서 반복되는 부분인 'li'를 타겟팅합니다)
    
    movies = soup.select(".movieBox-list .movieBox-item")

    print(f"현재 상영 중인 영화 {len(movies)}개를 발견했습니다.\n")

    for i, movie in enumerate(movies, 1):
        # 4. 제목 추출 
        # li 태그 내부에서 제목이 있는 클래스나 태그를 찾습니다.
        title_tag = movie.select_one("h3")
        title = title_tag.get_text(strip=True) if title_tag else "제목 정보 없음"

        # 5. 이미지 URL 추출
        # 작성하신 선택자의 마지막 부분인 'img'를 찾습니다.
        img_tag = movie.select_one("a > img")
        img_url = img_tag['src'] if img_tag else "이미지 없음"

        # 상대 경로 처리 (주소가 /로 시작하면 도메인 붙여주기)
        if img_url.startswith('/'):
            img_url = "https://www.moviechart.co.kr" + img_url

        print(f"{i}위: {title}")
        print(f"🔗 포스터 주소: {img_url}")
        print("-" * 50)

else:
    print(f"❌ 페이지 호출 실패! (에러 코드: {movie_ranking.status_code})")

✅ 페이지를 성공적으로 불러왔습니다!
현재 상영 중인 영화 20개를 발견했습니다.

1위: 왕과 사는 남자
🔗 포스터 주소: https://www.moviechart.co.kr/thumb?width=178&height=267&m_code=20242837&source=https://admin.moviechart.co.kr/assets/upload/movie/260108003526_8322.jpg
--------------------------------------------------
2위: 휴민트
🔗 포스터 주소: https://www.moviechart.co.kr/thumb?width=178&height=267&m_code=20241266&source=https://admin.moviechart.co.kr/assets/upload/movie/260112024651_6301.jpg
--------------------------------------------------
3위: 초속 5센티미터
🔗 포스터 주소: https://www.moviechart.co.kr/thumb?width=178&height=267&m_code=20259583&source=https://admin.moviechart.co.kr/assets/upload/movie/260213052538_7190.jpg
--------------------------------------------------
4위: 너자 2
🔗 포스터 주소: https://www.moviechart.co.kr/thumb?width=178&height=267&m_code=20261181&source=https://admin.moviechart.co.kr/assets/upload/movie/260202063756_4036.jpg
--------------------------------------------------
5위: 슬라이드 스트럼 뮤트
🔗 포스터 주소: https://www.moviechart.co.kr/thu

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd  # 판다스 불러오기

url = "https://www.moviechart.co.kr/rank/realtime/index/image"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

movie_ranking = requests.get(url, headers=headers)

if movie_ranking.status_code == 200:
    soup = BeautifulSoup(movie_ranking.content, 'html.parser')
    movies = soup.select(".movieBox-list .movieBox-item")
    
    # 1. 데이터를 담을 빈 리스트 준비
    movie_results = []

    for i, movie in enumerate(movies, 1):
        title_tag = movie.select_one("h3")
        title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

        img_tag = movie.select_one("a > img")
        img_url = img_tag['src'] if img_tag else "이미지 없음"
        if img_url.startswith('/'):
            img_url = "https://www.moviechart.co.kr" + img_url

        # 2. 딕셔너리 형태로 리스트에 추가
        movie_results.append({
            '순위': i,
            '영화제목': title,
            '포스터링크': img_url
        })

    # 3. 데이터프레임 생성
    df = pd.DataFrame(movie_results)

    # 결과 출력
    print(df)

    # # 4. 엑셀(CSV) 파일로 저장 (인코딩 설정 주의!)
    # df.to_csv("movie_rank.csv", index=False, encoding="utf-8-sig")
    # print("\n✅ movie_rank.csv 파일로 저장되었습니다!")

else:
    print("페이지 호출 실패")

    순위                   영화제목  \
0    1               왕과 사는 남자   
1    2                    휴민트   
2    3               초속 5센티미터   
3    4                   너자 2   
4    5            슬라이드 스트럼 뮤트   
5    6                    넘버원   
6    7                     햄넷   
7    8                     부흥   
8    9              매드 댄스 오피스   
9   10                   신의악단   
10  11           28년 후: 뼈의 사원   
11  12     렌탈 패밀리: 가족을 빌려드립니다   
12  13              몬테크리스토 백작   
13  14                    호퍼스   
14  15                 만약에 우리   
15  16                 프랑켄슈타인   
16  17                 폭풍의 언덕   
17  18                     점보   
18  19  아기 티라노 디보: 초식이지만 괜찮아!   
19  20              직사각형, 삼각형   

                                                포스터링크  
0   https://www.moviechart.co.kr/thumb?width=178&h...  
1   https://www.moviechart.co.kr/thumb?width=178&h...  
2   https://www.moviechart.co.kr/thumb?width=178&h...  
3   https://www.moviechart.co.kr/thumb?width=178&h...  
4   https://www.moviechart